In [749]:
import pandas as pd
from scipy.stats import skew
import numpy as np

In [750]:
# working on train and test filled in R with KNN filling

train = pd.read_csv('filled_train_knn.csv').drop(['Unnamed: 0'], axis= 1)
y = pd.read_csv('train.csv', usecols= ['Target_cost_euro'])


In [751]:
# removeing the outlier

data = pd.concat([y, train], axis=1)
i = data.loc[data["Target_cost_euro"] > 25000].index
data = data.drop(i)
i = data[data['Vehicle_max_speed_kmh'] > 300].index
data = data.drop(i)

In [752]:
test = pd.read_csv('filled_test_knn.csv').drop(['Unnamed: 0'], axis= 1)

In [753]:
# converting dates into int
train["Date_claim_opening"] = pd.to_numeric(train["Date_claim_opening"].str.replace('-', ''))
train["Date_of_accident"] = pd.to_numeric(train["Date_of_accident"].str.replace('-', ''))
train["Date_vehicle_immatriculation"] = pd.to_numeric(train["Date_vehicle_immatriculation"].str.replace('-', ''))

In [754]:
test["Date_claim_opening"] = pd.to_numeric(test["Date_claim_opening"].str.replace('-', ''))
test["Date_of_accident"] = pd.to_numeric(test["Date_of_accident"].str.replace('-', ''))
test["Date_vehicle_immatriculation"] = pd.to_numeric(test["Date_vehicle_immatriculation"].str.replace('-', ''))

In [755]:
#Dropping R columns
train = train.drop(['Date_claim_opening_imp', 'Date_of_accident_imp', 'Customer_code1_imp',
       'Customer_code3_imp', 'Customer_code4_imp', 'Customer_merit_class_imp',
       'Flag_virtuous_driver_imp', 'Presence_of_black_box_imp',
       'Bareme_table_code_customer_imp', 'Bareme_table_code_other_driver_imp',
       'Province_code_imp', 'Region_imp', 'Date_vehicle_immatriculation_imp',
       'Vehicle_typology_imp', 'Vehicle_power_source_imp',
       'Vehicle_catalytic_converter_imp','Flag_hybrid_vehicle_imp',
       'Flag_historical_vehicle_imp',
       'Vehicle_engine_capacity_imp', 'Vehicle_engine_rpm_imp',
       'Vehicle_torque_kgm_imp', 'Vehicle_torque_rpm_imp',
       'Vehicle_n_valves_cyl_imp', 'Vehicle_fiscal_HP_imp',
       'Vehicle_price_euro_imp', 'Vehicle_max_speed_kmh_imp',
       'Vehicle_acceleration_0_100_s_imp', 'Vehicle_length_m_imp',
       'Vehicle_width_m_imp', 'Vehicle_height_m_imp', 'Vehicle_weight_kg_imp',
       'Vehicle_weight_full_load_kg_imp', 'Vehicle_n_doors_imp',
       'Vehicle_n_seats_imp', 'Vehicle_code_supercharging_imp'], axis= 1)

test = test.drop(['Date_claim_opening_imp', 'Date_of_accident_imp', 'Customer_code1_imp',
       'Customer_code3_imp', 'Customer_code4_imp', 'Customer_merit_class_imp',
       'Flag_virtuous_driver_imp', 'Vehicle_annual_distance_km', 'Presence_of_black_box_imp',
       'Bareme_table_code_customer_imp', 'Bareme_table_code_other_driver_imp',
       'Province_code_imp', 'Region_imp', 'Date_vehicle_immatriculation_imp',
       'Vehicle_typology_imp', 'Vehicle_power_source_imp',
       'Vehicle_catalytic_converter_imp', 'Flag_hybrid_vehicle_imp',
       'Vehicle_annual_distance_km_imp', 'Flag_historical_vehicle_imp',
       'Vehicle_engine_capacity_imp', 'Vehicle_engine_rpm_imp',
       'Vehicle_torque_kgm_imp', 'Vehicle_torque_rpm_imp',
       'Vehicle_n_valves_cyl_imp', 'Vehicle_fiscal_HP_imp',
       'Vehicle_price_euro_imp', 'Vehicle_max_speed_kmh_imp',
       'Vehicle_acceleration_0_100_s_imp', 'Vehicle_length_m_imp',
       'Vehicle_width_m_imp', 'Vehicle_height_m_imp', 'Vehicle_weight_kg_imp',
       'Vehicle_weight_full_load_kg_imp', 'Vehicle_n_doors_imp',
       'Vehicle_n_seats_imp', 'Vehicle_code_supercharging_imp'], axis= 1)

In [757]:
#Bareme table

import itertools as it
bar = pd.read_csv('barome.csv', sep=';')
cust = list(bar.X)
bar.set_index('X', inplace=True)
other = list(bar.columns)

barome = dict()
for i in cust:
    for j in other:
        barome[(i,int(j))] = bar.loc[i][j]
        
l = []
for row in train.itertuples():
    c = row.Bareme_table_code_customer
    o = row.Bareme_table_code_other_driver
    l.append(barome[(c,o)])

    

train['Bareme_table_results'] = l

train["Bareme_table_results"] = train["Bareme_table_results"].map(dict(R=2, S=1, W=0))


In [758]:
# Bareme table

bar = pd.read_csv('barome.csv', sep=';')
cust = list(bar.X)
bar.set_index('X', inplace=True)
other = list(bar.columns)

barome = dict()
for i in cust:
    for j in other:
        barome[(i,int(j))] = bar.loc[i][j]
        
l = []
for row in test.itertuples():
    c = row.Bareme_table_code_customer
    o = row.Bareme_table_code_other_driver
    l.append(barome[(c,o)])
    
test['Bareme_table_results'] = l

test["Bareme_table_results"] = test["Bareme_table_results"].map(dict(R=2, S=1, W=0))


In [759]:
# log trasformation of skewed numeric features
numeric_feats = train.dtypes[train.dtypes != "object"].index

features = train[numeric_feats].apply(lambda x: skew(x.dropna())) 
features = features[features > 0.5]
print(features)
features = features.index
train[features] = np.log1p(train[features])

Customer_merit_class           2.532800
Presence_of_black_box          1.142703
Bareme_table_code_customer     4.338116
Vehicle_engine_capacity        1.537928
Vehicle_torque_kgm             1.102795
Vehicle_torque_rpm             0.690307
Vehicle_fiscal_HP              1.000214
Vehicle_price_euro             8.606305
Vehicle_max_speed_kmh          0.874647
Vehicle_height_m               0.916012
Vehicle_weight_kg              0.955111
Vehicle_weight_full_load_kg    0.810003
dtype: float64


In [760]:
# log trasformation of skewed numeric features
numeric_feats = test.dtypes[test.dtypes != "object"].index

features = test[numeric_feats].apply(lambda x: skew(x.dropna())) 
features = features[features > 0.5]
print(features)
features = features.index
test[features] = np.log1p(test[features])

Customer_merit_class           2.570352
Presence_of_black_box          1.173906
Bareme_table_code_customer     4.433789
Vehicle_engine_capacity        1.976828
Vehicle_torque_kgm             1.185350
Vehicle_torque_rpm             0.702365
Vehicle_fiscal_HP              1.225341
Vehicle_price_euro             2.243010
Vehicle_max_speed_kmh          0.911257
Vehicle_height_m               1.002216
Vehicle_weight_kg              0.948478
Vehicle_weight_full_load_kg    0.796702
dtype: float64


In [761]:
y = np.log1p(y)

In [762]:
#converting flags into binary [1,0]
train["Flag_virtuous_driver"] = train["Flag_virtuous_driver"].map(dict(S=1, N=0))
train["Flag_hybrid_vehicle"] = train["Flag_hybrid_vehicle"].map(dict(S=1, N=0))
train["Flag_historical_vehicle"] = train["Flag_historical_vehicle"].map(dict(S=1, N=0))

In [763]:
test["Flag_virtuous_driver"] = test["Flag_virtuous_driver"].map(dict(S=1, N=0))
test["Flag_hybrid_vehicle"] = test["Flag_hybrid_vehicle"].map(dict(S=1, N=0))
test["Flag_historical_vehicle"] = test["Flag_historical_vehicle"].map(dict(S=1, N=0))

# MODELS

In [764]:
ui = pd.get_dummies(pd.concat([train,test], ignore_index= True))

train = ui.iloc[:6775]
test = ui.iloc[6775:]

In [765]:
from sklearn.preprocessing import RobustScaler

train = RobustScaler().fit(train).transform(train)
test = RobustScaler().fit(test).transform(test)



In [656]:
import xgboost as xgb
from sklearn.kernel_ridge import KernelRidge

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor

In [657]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(train, y.values, test_size=0.00000000001, random_state=0)  


# XGBOOST

In [661]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 200, gamma = 0.4, random_state = 1)

grid_param = {  
    'alpha': [0.1,0.5,1]
    
}


gd_sr = GridSearchCV(estimator=xg_reg,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)

In [662]:
gd_sr.fit(X_train, y_train)  

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   25.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(alpha=10, base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0.4, learning_rate=0.1,
                                    max_delta_step=0, max_depth=2,
                                    min_child_weight=1, missing=None,
                                    n_estimators=200, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=1,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1, param_grid={'alpha': [0.1, 0.5, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=True)

In [663]:
print(gd_sr.best_params_)
gd_sr.best_score_

{'alpha': 0.1}


-0.49959952878477965

In [217]:
print(gd_sr.best_params_)
gd_sr.best_score_

{'colsample_bytree': 0.2, 'gamma': 0.5}


-0.49897525799369047

In [221]:
print(gd_sr.best_params_)
gd_sr.best_score_

{'colsample_bytree': 0.2, 'gamma': 0.5}


-0.49897525799369047

In [211]:
print(gd_sr.best_params_)
gd_sr.best_score_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}


-0.49996108979800696

In [61]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.3,
                max_depth = 1, alpha = 10, n_estimators = 200, random_state = 1)


In [62]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.3,
                max_depth = 1, alpha = 10, n_estimators = 200, random_state = 1)

xg_reg.fit(train, y)

preds = xg_reg.predict(test)
preds = np.expm1(preds)

In [63]:
r = open('submit2.txt', 'w')
for i in preds:
    r.write(str(i)+'\n')
    
    
r.close()

# RIDGE

In [33]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor


lasso = Lasso(alpha=0.0003, max_iter=50000, random_state=42)


In [68]:
grid_param = {  
    'alpha': [0.01,0.001, 0.0001],
}


gd_sr = GridSearchCV(estimator=lasso,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)


#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   16.6s finished


{'alpha': 0.001}


-0.517547345473462

In [77]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))


grid_param = {  
    'alpha': [0.01,0.001, 0.0005],
}



gd_sr = GridSearchCV(estimator=ENet,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)


#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter alpha for estimator Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('elasticnet',
                 ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True,
                            l1_ratio=0.9, max_iter=1000, normalize=False,
                            positive=False, precompute=False, random_state=3,
                            selection='cyclic', tol=0.0001,
                            warm_start=False))],
         verbose=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [139]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(n_estimators=500, learning_rate=0.05, 
                                    random_state =5)


grid_param = {  
        'learning_rate' : [0.2, 0.05],
        'n_estimators': [100, 200, 300],
        
}


gd_sr = GridSearchCV(estimator=GBoost,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)


#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   13.2s finished
C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.05, 'n_estimators': 300}


-0.5011506197535414

# GradientBoostingRegressor

In [87]:
GBoost = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=3, 
                                   max_features='sqrt', min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)


grid_param = {  
        'max_depth': [3],
        'learning_rate' : [0.05],
        'n_estimators': [500],
        'min_samples_split' : [10,15],
        'loss' : ['ls', 'lad', 'huber', 'quantile']
        
}


gd_sr = GridSearchCV(estimator=GBoost,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)


#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   52.3s finished
C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 500}


-0.5018055511440901

In [ ]:
from sklearn import svm

clf = svm.SVR(kernel='linear') # Linear Kernel

grid_param = {  
    'kernel': ['linear', 'poly', 'rbf'],
}


gd_sr = GridSearchCV(estimator=clf,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)


#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [20]:
from sklearn.ensemble import RandomForestRegressor

rfr=RandomForestRegressor(n_estimators=100)

grid_param = {  
    'max_depth': [1,2],
    'n_estimators': [1000]
}


gd_sr = GridSearchCV(estimator=rfr,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     verbose = True,
                     n_jobs=-1)

#Train the model using the training sets
gd_sr.fit(X_train, y_train)
print(gd_sr.best_params_)
gd_sr.best_score_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   54.9s remaining:   36.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.1min finished
C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'max_depth': 2, 'n_estimators': 1000}


-0.5129295419376225

In [206]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 200, colsample_bytree: 0.2, gamma: 0.5, random_state = 1)

lasso = Lasso(alpha=0.0003, max_iter=50000, random_state=42)

GBoost = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=3, 
                                   max_features='sqrt', min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

ada = AdaBoostRegressor(n_estimators=200, learning_rate=0.1, 
                                    random_state =5)


KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

xg_reg.fit(train, y)
xg_preds = np.expm1(xg_reg.predict(test))

ada.fit(train, y)
ada_preds = np.expm1(ada.predict(test))


lasso.fit(train, y)
lasso_preds = np.expm1(lasso.predict(test))


GBoost.fit(train, y)
gboost_preds = np.expm1(GBoost.predict(test))



C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [200]:
def get_tuples(length, total):
    if length == 1:
        yield (total,)
        return
    
    for i in range(total + 1):
        for t in get_tuples(length - 1, total - i):
            yield (i,) + t

In [201]:
from sklearn.metrics import mean_absolute_error


X_train, X_test, y_train, y_test = train_test_split(train, y.values, test_size=0.2, random_state=0)  
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 200, random_state = 1)

lasso = Lasso(alpha=0.0003, max_iter=50000, random_state=42)

GBoost = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=3, 
                                   max_features='sqrt', min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

ada = AdaBoostRegressor(n_estimators=200, learning_rate=0.1, 
                                    random_state =5)

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

ada.fit(X_train, y_train)
ada_preds = np.expm1(ada.predict(X_test))

xg_reg.fit(X_train, y_train)
xg_preds = np.expm1(xg_reg.predict(X_test))

lasso.fit(X_train, y_train)
lasso_preds = np.expm1(lasso.predict(X_test))


GBoost.fit(X_train, y_train)
gboost_preds = np.expm1(GBoost.predict(X_test))





C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mccol\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [202]:
temp = 1000000
tu = () 
for i in get_tuples(4,100):
    preds = (i[0]/100)* xg_preds + (i[1]/100) *  ada_preds + (i[2]/100) *  gboost_preds + (i[3]/100) *  lasso_preds
    if mean_absolute_error(np.expm1(y_test),  preds) < temp:
        temp = mean_absolute_error(np.expm1(y_test),  preds)
        tu = i
        
        

In [766]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1,
                max_depth = 2, alpha = 0.01, n_estimators = 200,  gamma = 0.4, random_state = 1)

xg_reg.fit(train, y)
xg_preds = np.expm1(xg_reg.predict(test))


In [767]:
preds = xg_preds

In [768]:
r = open('submit2.txt', 'w')
for i in preds:
    r.write(str(i)+'\n')
     
    
r.close()

In [769]:
preds = xg_preds
preds = pd.DataFrame(preds, columns= ['fin'])

In [795]:
q1 = preds['fin'].quantile(0.0004)
q2 = preds['fin'].quantile(0.99)

preds['fin'] = preds['fin'].apply(lambda x: x if x > q1 else x*1.4)
preds['fin'] = preds['fin'].apply(lambda x: x if x < q2 else x*1.4)

In [800]:
preds = xg_preds
preds = pd.DataFrame(preds, columns= ['fin'])

q1 = preds['fin'].quantile(0.0004)
q2 = preds['fin'].quantile(0.99)

preds['fin'] = preds['fin'].apply(lambda x: x if x > q1 else x*1.45)
preds['fin'] = preds['fin'].apply(lambda x: x if x < q2 else x*1.25)
r = open('submit3.txt', 'w')

for i in preds.index:
    print(preds.iloc[i][0])
    r.write(str(preds.iloc[i][0])+'\n')
    
r.close()

911.692138671875
763.2453002929688
1037.713134765625
887.4872436523438
1084.99560546875
979.6470947265625
1150.0264892578125
953.4028930664062
814.587646484375
1021.0853271484375
929.9578857421875
1204.530029296875
1058.2918701171875
1172.8824462890625
939.17138671875
894.2345581054688
1067.1427001953125
1063.271728515625
1680.1666259765625
1584.045166015625
1236.9901123046875
895.4691162109375
1204.2208251953125
1065.46435546875
827.5751342773438
1183.933837890625
1858.0015869140625
984.0736694335938
969.5486450195312
1038.341796875
739.620361328125
1099.03369140625
1189.822265625
901.7095336914062
1610.6827392578125
811.7070922851562
1078.9110107421875
1122.158447265625
1101.7762451171875
919.6350708007812
1027.138671875
820.6802978515625
1015.4556884765625
1221.8380126953125
671.97216796875
1130.7052001953125
996.1141357421875
904.1595458984375
1200.460693359375
1679.055908203125
1432.66259765625
1172.508544921875
1548.5816650390625
1095.3916015625
1082.551025390625
968.130249023437

1252.1966552734375
1034.4991455078125
1215.807373046875
1056.9505615234375
1347.7879638671875
1022.3888549804688
832.8071899414062
1383.934326171875
1223.7496337890625
1060.4761962890625
973.6292724609375
987.2479248046875
1157.710205078125
910.4978637695312
1706.395263671875
923.6478271484375
944.2464599609375
1422.630126953125
899.8592529296875
1106.22119140625
771.111083984375
1110.637451171875
948.6707153320312
943.3031005859375
1763.2998046875
1273.70458984375
1704.26025390625
1280.744384765625
1012.3975830078125
904.01318359375
1010.0672607421875
877.1871337890625
1118.7840576171875
790.7285766601562
1106.4873046875
935.2684936523438
882.8981323242188
1719.2310791015625
991.8986206054688
1179.2218017578125
902.7897338867188
887.1407470703125
986.3285522460938
1078.18359375
1094.9080810546875
1709.8988037109375
1339.861328125
1168.9439697265625
1398.5106201171875
1302.11962890625
942.1596069335938
1033.0914306640625
930.4699096679688
834.4675903320312
882.3878784179688
840.0872802

1179.0130615234375
950.8563842773438
1045.579345703125
1386.2740478515625
1074.9521484375
1336.56298828125
944.99853515625
937.178955078125
997.20263671875
1001.1892700195312
933.1912231445312
1017.2685546875
1012.7600708007812
1691.6112060546875
846.1146850585938
898.7872314453125
1083.4432373046875
913.94140625
961.0136108398438
1183.6812744140625
1179.5511474609375
954.5130615234375
1204.841064453125
1223.4261474609375
1075.8118896484375
1049.5147705078125
1016.0389404296875
1312.06201171875
1270.0477294921875
1093.3446044921875
1376.2003173828125
920.2911376953125
1130.4068603515625
1144.37841796875
822.8954467773438
1341.7154541015625
987.8135986328125
1289.2490234375
1160.5103759765625
749.275146484375
1109.68896484375
1085.6358642578125
1075.7662353515625
741.1751708984375
1041.4871826171875
853.5182495117188
1149.21728515625
1576.100341796875
1217.2459716796875
1607.4306640625
1015.47314453125
899.35595703125
1010.8143310546875
1099.5069580078125
980.2102661132812
1141.78283691

851.3406982421875
1512.6395263671875
930.1741333007812
1038.743408203125
1782.19873046875
1452.870849609375
1000.40966796875
1273.64453125
1305.69921875
1659.2728271484375
935.9678344726562
895.22802734375
1808.6976318359375
822.96142578125
996.3947143554688
1186.185302734375
2535.1425170898438
1025.4061279296875
1024.4306640625
1042.7296142578125
1235.5670166015625
855.1166381835938
1084.424072265625
933.2023315429688
1042.01123046875
1104.0418701171875
1180.4482421875
1264.16357421875
1126.2269287109375
1193.9359130859375
791.8173828125
1319.6712646484375
1138.6407470703125
1334.492431640625
1203.2672119140625
1495.4395751953125
1111.54736328125
1100.743408203125
805.0223388671875
1141.9827880859375
937.95458984375
898.8867797851562
1158.618896484375
1933.4462890625
1019.5614624023438
868.9924926757812
1046.7972412109375
1140.6646728515625
881.9267578125
1097.26904296875
1158.7052001953125
1722.4495849609375
1044.33544921875
1218.7293701171875
923.0867309570312
1272.184814453125
1054

1113.895263671875
1247.0406494140625
1148.9068603515625
1196.564453125
923.9525146484375
1141.728271484375
1064.442626953125
1852.57470703125
981.421875
882.8167724609375
1219.6783447265625
1091.403076171875
1095.9129638671875
861.4318237304688
1182.3934326171875
914.2874755859375
1098.4427490234375
996.8271484375
1156.203369140625
1657.017333984375
984.4217529296875
1093.51953125
1222.07763671875
983.4782104492188
1095.87158203125
942.236083984375
1593.4798583984375
846.1558837890625
914.8921508789062
952.522216796875
1145.6292724609375
1105.45166015625
1427.218994140625
967.4756469726562
1034.5164794921875
971.5564575195312
1469.6221923828125
1034.87255859375
1359.3599853515625
888.7446899414062
971.254150390625
1892.19287109375
1052.935546875
1171.0316162109375
899.8575439453125
1063.388427734375
896.9605102539062
1567.8565673828125
1010.8143310546875
1112.842041015625
1197.6435546875
1099.4010009765625
1156.7696533203125
1499.561767578125
1038.20947265625
1064.7135009765625
942.728